In [ ]:
import torch
import itertools

In [ ]:
t = torch.rand((2, 10)).cuda()
print(t)

In [ ]:
a = torch.load('/root/secret_sauce/nggyu22000-compressed.pt', map_location='cpu')



In [ ]:
len(t), len(t[0])

In [ ]:
t[0].shape

In [ ]:
tt = list(itertools.chain(*t))
a = torch.stack(tt).squeeze(1).squeeze(1).view(-1).type(torch.int16)

In [ ]:
a.shape,a[8250], a.dtype

In [ ]:
torch.save(a.cpu().numpy(), f='./nggyu22000-compressed.pt')

In [ ]:
from secret_sauce.network.basic_transformer.transformer import PositionalEncoding
pos = PositionalEncoding(64, max_len=600000)

In [ ]:
content = torch.tensor(a[0:3]).unsqueeze(1).expand(3, 64).unsqueeze(0).type(torch.int)
positions = torch.tensor([0,1,2], dtype=torch.long).unsqueeze(0)

content.shape, positions.shape




In [ ]:
import math
pe = torch.zeros(600000, 64)
position = torch.arange(0, 600000, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, 64, 2).float() * (-math.log(10000.0) / 64))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)


positions # [B T]

pe.shape # [T F]

(content + pe[positions  ,:])


In [ ]:
pos(content, positions)


In [ ]:
from secret_sauce.network.basic_transformer.transformer import BasicTransformer
from secret_sauce.config.config import Config
cfg = Config()
cfg

In [ ]:
transformer = BasicTransformer(cfg)
transformer.load_embeddings(embed_wts)

In [ ]:
mask = transformer.generate_square_subsequent_mask(3)
content = content[:, :, 0]
content.shape, positions.shape, mask.shape


In [ ]:
transformer(content, positions, mask)


In [ ]:
transformer.pos_encoder.pe[positions, :]


In [ ]:
t = torch.load(r'/root/secret_sauce/outputs/04-16-2021-10-58-09/epoch-900/mp_rank_00_model_states.pt', map_location='cpu')

In [ ]:
embed_wts = t['module']['vector_quantizer.embedding.weight']


In [ ]:
embed_wts.shape, embed_wts.device, embed_wts[0]